In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Strategy we will backtest

#### 1. At 10:59 AM calculate the day high and day Low.
    
    
#### 2. From 11 AM keep monitoring the stock price.


#### 3. If it breaches the Low enter the trade with Short


#### 4. If it breaches the high enter the trade with Long


#### 5. If neither of #4 or #5 happens, don't enter.


#### 6. Once, trade entered. Set yourself a target and a stop loss.


#### 7. Exit the trade when either target or stop-loss is breached.

## Step 0 - Declaring the Parameters

In [2]:
# To be used in Profit and Loss calculcation
brokerage = 40

# Quantity of stocks in 1 Lot of Bank Nifty
lot_size = 25

# This is a factor we are including for slippage and other tax deductions
# This variable is to incorporate STT deductions
adjustment_factor = 0.95 

# Interval for which we calucalate our High and Low
interval_dict = {"10 AM" :  45,
                 "11 AM" :  105,
                 "12 PM" :  165,
                 "1 PM"  :  225,
                 "2 PM"  :  285}


interval_in_mins = interval_dict['11 AM']

start_time = '10:59' #change it as per interval selection
end_time = '15:00'

#Target Points
target_param = 100

# Stop Loss Points
loss_param = 175

# This date is an outlier. On this day Trading happens at a different time-frame.
muhurat_trading_day = '04-11-2021' 


#File Path
# data_path = '/Users/sreemantakesh/Desktop/The Algotrading Page/Datasets/FUTURES/BANK NIFTY FUT 2021.csv'
data_path = "C:/Users/ganesh/Desktop/BANK NIFTY FUT 2021.csv"

## Step 1 - Load the data and perform elementary checks

#### 1. No missing values.

#### 2. Check the data-types

In [3]:
df = pd.read_csv(data_path)
df.head(5)

,ticker,date,open,high,low,close,volume,open interest,month
0,BANKNIFTY28JAN21FUT.NFO,04-01-2021 09:15,31450.25,31540.00,31450.05,31498.35,74575,1433225.0,JAN
1,BANKNIFTY28JAN21FUT.NFO,04-01-2021 09:16,31499.00,31511.45,31484.25,31498.00,41275,1433225.0,JAN
2,BANKNIFTY28JAN21FUT.NFO,04-01-2021 09:17,31491.55,31504.85,31475.00,31484.95,21050,1465875.0,JAN
3,BANKNIFTY28JAN21FUT.NFO,04-01-2021 09:18,31485.10,31512.55,31485.00,31509.00,26650,1465875.0,JAN
4,BANKNIFTY28JAN21FUT.NFO,04-01-2021 09:19,31512.35,31530.00,31512.35,31530.00,43300,1465875.0,JAN


In [4]:
df.isnull().sum()

ticker              0
date                0
open                0
high                0
low                 0
close               0
volume              0
open interest    1504
month               0
dtype: int64

In [5]:
df.dtypes

ticker            object
date              object
open             float64
high             float64
low              float64
close            float64
volume             int64
open interest    float64
month             object
dtype: object

# Step 2 - Create two columns time, and day. Such that

#### 'Day' = DD-MM-YYYY
#### 'Time' = HH:MM
    
Also, convert, the 'date' column to 'datetime' object.

In [6]:
df['time']= df.date.str.slice(10,16)

df['day'] = df.date.str.slice(0,10)

df["date"] = pd.to_datetime(df["date"])

df.head(5)

,ticker,date,open,high,low,close,volume,open interest,month,time,day
0,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:15:00,31450.25,31540.00,31450.05,31498.35,74575,1433225.0,JAN,09:15,04-01-2021
1,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:16:00,31499.00,31511.45,31484.25,31498.00,41275,1433225.0,JAN,09:16,04-01-2021
2,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:17:00,31491.55,31504.85,31475.00,31484.95,21050,1465875.0,JAN,09:17,04-01-2021
3,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:18:00,31485.10,31512.55,31485.00,31509.00,26650,1465875.0,JAN,09:18,04-01-2021
4,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:19:00,31512.35,31530.00,31512.35,31530.00,43300,1465875.0,JAN,09:19,04-01-2021


# Step 3 - At 10:59 AM calculate the day high and day Low  for each day and store them as columns with name as 'interval_high' and 'interval_low'. 

In [7]:
#doing a groupby and only keeping the first 105 entries. Why?
temp = df.groupby('day').head(interval_in_mins)
temp.head(5)

,ticker,date,open,high,low,close,volume,open interest,month,time,day
0,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:15:00,31450.25,31540.00,31450.05,31498.35,74575,1433225.0,JAN,09:15,04-01-2021
1,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:16:00,31499.00,31511.45,31484.25,31498.00,41275,1433225.0,JAN,09:16,04-01-2021
2,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:17:00,31491.55,31504.85,31475.00,31484.95,21050,1465875.0,JAN,09:17,04-01-2021
3,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:18:00,31485.10,31512.55,31485.00,31509.00,26650,1465875.0,JAN,09:18,04-01-2021
4,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:19:00,31512.35,31530.00,31512.35,31530.00,43300,1465875.0,JAN,09:19,04-01-2021


In [8]:
interval_high_low = temp.groupby('day', as_index=False).agg({"high":"max", "low":"min"})
interval_high_low.head(5)

,day,high,low
0,01-02-2021,31333.80,30942.50
1,01-03-2021,35615.00,35009.00
2,01-04-2021,33750.00,33390.05
3,01-06-2021,35797.45,35360.00
4,01-07-2021,34996.95,34810.85


In [9]:
#renaming the columns
interval_high_low.columns = ['day','interval_high','interval_low']

#Joining with the original dataframe 'df'
df = pd.merge(df, interval_high_low, on = 'day')

df.head(5)

,ticker,date,open,high,low,close,volume,open interest,month,time,day,interval_high,interval_low
0,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:15:00,31450.25,31540.00,31450.05,31498.35,74575,1433225.0,JAN,09:15,04-01-2021,31599.0,31200.0
1,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:16:00,31499.00,31511.45,31484.25,31498.00,41275,1433225.0,JAN,09:16,04-01-2021,31599.0,31200.0
2,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:17:00,31491.55,31504.85,31475.00,31484.95,21050,1465875.0,JAN,09:17,04-01-2021,31599.0,31200.0
3,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:18:00,31485.10,31512.55,31485.00,31509.00,26650,1465875.0,JAN,09:18,04-01-2021,31599.0,31200.0
4,BANKNIFTY28JAN21FUT.NFO,2021-04-01 09:19:00,31512.35,31530.00,31512.35,31530.00,43300,1465875.0,JAN,09:19,04-01-2021,31599.0,31200.0


# Step 4 - Filter the data for just one day '04-01-2021' . And do the following.

#### 1. Store the 11 AM high and 11 AM low in variables 'interval_high' and 'interval_low'

#### 2. Store the 11 AM high and 11 AM low in variables 'interval_high_time' and 'interval_low_time'

#### 3. Filter the data further such that we only have data from 11 AM to 3 PM for the given day/

In [10]:
day = '04-01-2021'

In [11]:
#filtering the data for one single day
d1 = df[df["day"] == day]

#Storing the 11 AM high and 11 AM low in variables 'interval_high' and 'interval_low'
interval_high = d1['interval_high'].unique()[0]
interval_low = d1['interval_low'].unique()[0]

#Storing the 11 AM high and 11 AM low in variables 'interval_high_time' and 'interval_low_time'
interval_high_time = d1[d1['high']==interval_high].date.dt.time.values[0]
interval_low_time = d1[d1['low']==interval_low].date.dt.time.values[0]
    
#subsetting the data for post to interval time
d1 = d1.set_index('date')
d1 = d1.between_time(start_time,end_time)
d1 = d1.reset_index()

d1.head(5)

,date,ticker,open,high,low,close,volume,open interest,month,time,day,interval_high,interval_low
0,2021-04-01 10:59:00,BANKNIFTY28JAN21FUT.NFO,31235.2,31275.0,31235.00,31246.4,29400,1635600.0,JAN,10:59,04-01-2021,31599.0,31200.0
1,2021-04-01 11:00:00,BANKNIFTY28JAN21FUT.NFO,31233.0,31233.0,31056.15,31102.0,162475,1635600.0,JAN,11:00,04-01-2021,31599.0,31200.0
2,2021-04-01 11:01:00,BANKNIFTY28JAN21FUT.NFO,31113.8,31148.0,31076.85,31148.0,58250,1635600.0,JAN,11:01,04-01-2021,31599.0,31200.0
3,2021-04-01 11:02:00,BANKNIFTY28JAN21FUT.NFO,31149.9,31176.2,31144.00,31170.0,41800,1650250.0,JAN,11:02,04-01-2021,31599.0,31200.0
4,2021-04-01 11:03:00,BANKNIFTY28JAN21FUT.NFO,31172.1,31172.1,31128.00,31130.0,29225,1650250.0,JAN,11:03,04-01-2021,31599.0,31200.0


# Step 5 - Check if trade is triggered or not.

#### 1. Create two boolean columns 'interval_high_breach' and 'interval_low_breach' such that position after the high breach or low breach are all marked as 1.

#### 2. Declare and empty variable name it as 'position'

#### 3. Check if trade is triggered or not. If it breaks high than store position = 'Long', if low is breached then store as 'Short' else "NA"

#### 4. Store the 'row number' or the index of the dataframe where the trade was triggered in a variable 'entry_index' or 'entry_index'

#### 4. Also, store the entry_price, target, stop_loss, entry_time for all the three cases

In [12]:
d1.columns

Index(['date', 'ticker', 'open', 'high', 'low', 'close', 'volume',
       'open interest', 'month', 'time', 'day', 'interval_high',
       'interval_low'],
      dtype='object')

In [13]:
position = ''

d1['interval_high_breach'] = np.where(d1.high > interval_high , True, False)
d1['interval_low_breach'] = np.where(d1.low < interval_low , True, False)
d1[['time', 'day', 'open', 'high', 'low', 'close','time', 'day', 'interval_high','interval_low', 'interval_high_breach', 'interval_low_breach']]

,time,day,open,high,low,close,time,day,interval_high,interval_low,interval_high_breach,interval_low_breach
0,10:59,04-01-2021,31235.20,31275.0,31235.00,31246.40,10:59,04-01-2021,31599.0,31200.0,False,False
1,11:00,04-01-2021,31233.00,31233.0,31056.15,31102.00,11:00,04-01-2021,31599.0,31200.0,False,True
2,11:01,04-01-2021,31113.80,31148.0,31076.85,31148.00,11:01,04-01-2021,31599.0,31200.0,False,True
3,11:02,04-01-2021,31149.90,31176.2,31144.00,31170.00,11:02,04-01-2021,31599.0,31200.0,False,True
4,11:03,04-01-2021,31172.10,31172.1,31128.00,31130.00,11:03,04-01-2021,31599.0,31200.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
237,14:56,04-01-2021,31250.00,31260.0,31248.00,31260.00,14:56,04-01-2021,31599.0,31200.0,False,False
238,14:57,04-01-2021,31255.45,31270.0,31254.00,31262.00,14:57,04-01-2021,31599.0,31200.0,False,False
239,14:58,04-01-2021,31263.00,31263.0,31255.00,31260.00,14:58,04-01-2021,31599.0,31200.0,False,False
240,14:59,04-01-2021,31255.90,31265.0,31254.00,31257.55,14:59,04-01-2021,31599.0,31200.0,False,False


In [14]:
#Condtion 1 - When after 11 AM both high breach and low breack has happend on that day

if len(d1[d1['interval_low_breach'] == True]) & len(d1[d1['interval_high_breach'] == True]):
    
    long_entry_index = d1[d1['interval_high_breach'] == True].index[0]
    short_entry_index = d1[d1['interval_low_breach'] == True].index[0]


    if long_entry_index < short_entry_index:

        position = 'Long'
        
        entry_price = interval_high
        
        entry_index = d1[d1['interval_high_breach'] == True].index[0]
        
        target    = entry_price + target_param
        
        stop_loss = entry_price - loss_param
        
        entry_time = d1['date'].dt.time[entry_index]    
        
        
    else:

        position = 'Short'
        
        entry_price = interval_low
        
        entry_index = d1[d1['interval_low_breach'] == True].index[0]
        
        target    = entry_price - target_param
        
        stop_loss = entry_price + loss_param
        
        entry_time = d1['date'].dt.time[entry_index]
        
#Condtion 2 - When after 11 AM only low breach has happened
elif len(d1[d1['interval_low_breach'] == True]):

    position = 'Short'

    entry_price = interval_low

    entry_index = d1[d1['interval_low_breach'] == True].index[0]

    target    = entry_price - target_param

    stop_loss = entry_price + loss_param

    entry_time = d1['date'].dt.time[entry_index]

#Condtion 3 - When after 11 AM only high breach has happened
elif len(d1[d1['interval_high_breach'] == True]):

    position = 'Long'

    entry_price = interval_high

    entry_index = d1[d1['interval_high_breach'] == True].index[0]

    target    = entry_price + target_param

    stop_loss = entry_price - loss_param

    entry_time = d1['date'].dt.time[entry_index]   
        
else:

    position = 'NA'
    
    entry_price = np.nan

    entry_index = np.nan

    target    = np.nan

    stop_loss = np.nan

    entry_time = np.nan

In [15]:
print("Trade triggered {} position entered ".format(position))
print("Trade entered at : ",format(entry_time))
print("Entry Price : ",entry_price)
print("Traget : ",target)
print("Stop Loss : ",stop_loss)

Trade triggered Short position entered 
Trade entered at :  11:00:00
Entry Price :  31200.0
Traget :  31100.0
Stop Loss :  31375.0


# Step 6 : Check the outcome of the Trade
    
#### 1. Declare two empty variables 'closing_price' and 'exit_time'

#### 2. Loop through the data after the trade has entered to check for profit or loss.

#### 3. Store the outcome in variable called 'result'. Also store the exit time and closing price 

In [16]:
closing_price = ''
exit_time = ''

if position!="NA":

    post_signal_data = d1.iloc[entry_index:,:][["high","low","close","time"]].reset_index(drop=True).copy()        

    #iterate and check if target hits or stop loss hits
    if position == "Long":
        for candle_high, candle_low, closing,time in zip(post_signal_data.high,
                                                         post_signal_data.low,
                                                         post_signal_data.close,
                                                         post_signal_data.time):
            if candle_high > target:
                result = 'Profit'
                #print('Profit')
                closing_price = target
                exit_time = time
                break
            elif candle_low < stop_loss:
                result = 'Loss'
                #print('Loss')
                closing_price = stop_loss
                exit_time = time
                break
            else:
                result = "Started but not finished"
                closing_price = closing
                exit_time = time

    else:
        for candle_high, candle_low, closing, time in zip(post_signal_data.high,
                                                          post_signal_data.low,
                                                          post_signal_data.close,
                                                          post_signal_data.time):
            if candle_low < target:
                result = 'Profit'
                closing_price = target
                exit_time = time
                #print('Profit')
                break
            elif candle_high > stop_loss:
                result = 'Loss'
                closing_price = stop_loss
                exit_time = time
                #print('Loss')
                break
            else:
                result = "Started but not finished"
                last_price = post_signal_data
                closing_price = closing
                exit_time = time
else:
    result = "NA"
    closing_price = 'NA'
    
# Copy the data in a list
rpt = []

rpt.append((day,
            interval_high,
            interval_high_time,
            interval_low,
            interval_low_time,
            position,
            entry_time,
            exit_time,
            result,
            entry_price,
            closing_price))

In [17]:
print("Trade triggered {} position entered ".format(position))
print("Trade entered at : ",format(entry_time))
print("Entry Price : ",entry_price)
print("Traget : ",target)
print("Stop Loss : ",stop_loss)
print("Trade Result : ",result)
print("Trade Exited at : ",exit_time)

Trade triggered Short position entered 
Trade entered at :  11:00:00
Entry Price :  31200.0
Traget :  31100.0
Stop Loss :  31375.0
Trade Result :  Profit
Trade Exited at :   11:00


# Step 7 : Store the Result in the form of a DataFrame

In [18]:
rpt_df = pd.DataFrame(rpt,columns= ['day',
                'interval_high',
                'interval_high_time',
                'interval_low',
                'interval_low_time',
                'position',
                'entry_time',
                'exit_time',
                'result',
                'entry_price',
                'closing_price'])

rpt_df

,day,interval_high,interval_high_time,interval_low,interval_low_time,position,entry_time,exit_time,result,entry_price,closing_price
0,04-01-2021,31599.0,09:28:00,31200.0,10:57:00,Short,11:00:00,11:00,Profit,31200.0,31100.0
